## _*Using Qiskit Aqua for stable-set problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE algorithm to compute the maximum stable set of a given graph.  

The problem is defined as follows. Given a graph $G = (V,E)$, we want to compute $S \subseteq V$ such that there do not exist $i, j \in S : (i, j) \in E$, and $|S|$ is maximized. In other words, we are looking for a maximum cardinality set of mutually non-adjacent vertices.

The graph provided as an input is used first to generate an Ising Hamiltonian, which is then passed as an input to VQE.  As a reference, this notebook also computes the maximum stable set using the Exact Eigensolver classical algorithm and the solver embedded in the commercial IBM CPLEX product (if it is available in the system and the user has followed the necessary configuration steps in order for Qiskit Aqua to find it).  Please refer to the Qiskit Aqua Optimization documentation for installation and configuration details for CPLEX.

In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.optimization.ising import stable_set
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.optimization.ising.common import parse_gset_format, random_graph, sample_most_likely

Here an Operator instance is created for our Hamiltonian. In this case the Paulis are from an Ising Hamiltonian of the maximum stable set problem (expressed in minimization form). We load a small instance of the maximum stable set problem.

In [2]:
w = parse_gset_format('sample.maxcut')
qubitOp, offset = stable_set.get_operator(w)

We also offer a function to generate a random graph as a input.

In [3]:
if True:
    np.random.seed(8123179)
    w = random_graph(5, edge_prob=0.5)
    qubitOp, offset = stable_set.get_operator(w)
print(w)

[[ 0.  8. -9.  0.  7.]
 [ 8.  0.  9. -8. -1.]
 [-9.  9.  0.  0. -5.]
 [ 0. -8.  0.  0.  0.]
 [ 7. -1. -5.  0.  0.]]


Here we test for the presence of algorithms we want to use in this notebook. If Aqua is installed correctly `ExactEigensolver` and `VQE` will always be found. `CPLEX.Ising` is dependent on IBM CPLEX being installed (see introduction above). CPLEX is *not required* but if installed then this notebook will demonstrate the `CPLEX.Ising` algorithm , that uses CPLEX, to compute stable set as well.

In [4]:
to_be_tested_algos = ['ExactEigensolver', 'CPLEX.Ising', 'VQE']
print(to_be_tested_algos)

['ExactEigensolver', 'CPLEX.Ising', 'VQE']


We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. The result is a dictionary.

In [5]:
result = ExactEigensolver(qubitOp).run()
x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('stable set objective:', result['energy'] + offset)
print('solution:', stable_set.get_graph_solution(x))
print('solution objective and feasibility:', stable_set.stable_set_value(x, w))

energy: -29.5
stable set objective: -25.0
solution: [0. 0. 1. 1. 1.]
solution objective and feasibility: (3.0, False)


*Note*: IBM CPLEX is an _optional_ installation addition for Aqua. If installed then the Aqua CPLEX.Ising algorithm will be able to be used. If not, then solving this problem using this particular algorithm will simply be skipped.

We change the configuration parameters to solve it with the CPLEX backend. The CPLEX backend can deal with a particular type of Hamiltonian called Ising Hamiltonian, which consists of only Pauli Z at most second order and can be used for combinatorial optimization problems that can be formulated as quadratic unconstrained binary optimization problems, such as the stable set problem. Note that we may obtain a different solution - but if the objective value is the same as above, the solution will be optimal.

In [6]:
try:
    from qiskit.aqua.algorithms import CPLEX_Ising
    result = CPLEX_Ising(qubitOp, display=0).run()

    x_dict = result['x_sol']
    print('energy:', result['energy'])
    print('time:', result['eval_time'])
    print('stable set objective:', result['energy'] + offset)
    x = np.array([x_dict[i] for i in sorted(x_dict.keys())])
    print('solution:', stable_set.get_graph_solution(x))
    print('solution objective and feasibility:', stable_set.stable_set_value(x, w))
except Exception as ex:
    print(str(ex))

CPLEX is not installed. See https://www.ibm.com/support/knowledgecenter/SSSA5P_12.8.0/ilog.odms.studio.help/Optimization_Studio/topics/COS_home.html


Now we want VQE and so change it and add its other configuration parameters.

In [7]:
vqe = VQE(qubitOp,
          RYRZ(qubitOp.num_qubits, depth=3, entanglement='linear'),
          L_BFGS_B(maxfun=6000))
result = vqe.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('stable set objective:', result['energy'] + offset)
print('solution:', stable_set.get_graph_solution(x))
print('solution objective and feasibility:', stable_set.stable_set_value(x, w))

energy: -29.49999999993873
time: 26.565035104751587
stable set objective: -24.99999999993873
solution: [0. 0. 1. 1. 1.]
solution objective and feasibility: (3.0, False)
